In [1]:
import numpy as np
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf
import datetime
import argparse
#parser = argparse.ArgumentParser()
#parser.add_argument('--train_dir', type=str, default='./trainDataHJ.txt')
#parser.add_argument('--rules_dir', type=str, default='./rulesDataHJ.txt')
#parser.add_argument('--support', type=float, default=0.05)
#parser.add_argument('--confidence',type=float,default=0.2)
#args = parser.parse_args()

In [2]:
y=np.random.randint(1,6,[1000,])
hum=np.random.randint(0,101,[1000,])
tem=np.random.uniform(-50.0,40.0,[1000,])
is_gas=np.random.randint(0,2,1000)
per_con=np.random.randint(0,101,[1000,])
is_metal=np.random.randint(0,2,1000)
mach_cat=np.random.randint(1,4,[1000,])
meth_cat=np.random.randint(1,4,[1000,])


In [3]:
df=pd.DataFrame({
    "hum":hum,
    "tem":tem,
    "is_gas":is_gas,
    "per_con":per_con,
    "is_metal":is_metal,
    "mach_cat":mach_cat,
    "meth_cat":meth_cat,
    "time":y
})

In [5]:
#df.to_csv('trainDataHJ.csv',index=False)

In [6]:
def data_pre(path):
    df=pd.read_csv(path)
    #iscretization
    bins=[0,50,70,90,100]
    df['hum']=pd.cut(df['hum'],bins,right=False,labels=['1','2','3','4'])
    bins=[-50.0,-20.0,-10.0,0,10.0,20.0,30.0,41.0]
    df['tem']=pd.cut(df['tem'],bins,right=False,labels=['1','2','3','4','5','6','7'])
    bins=[0,20,100]
    df['per_con']=pd.cut(df['per_con'],bins,right=False,labels=['1','2'])
    #type trans
    df['is_gas']=df['is_gas'].apply(str)
    df['is_metal']=df['is_metal'].apply(str)
    df['mach_cat']=df['mach_cat'].apply(str)
    df['meth_cat']=df['meth_cat'].apply(str)
    df['time']=df['time'].apply(str)
    #one_hot
    df=pd.get_dummies(df)
    strDecode=df.columns.values.tolist()
    data=df.values
    data.dtype=bool
    return data,strDecode

In [7]:
def model(data,support=0.05,confidence=0.2):
    fre_ite=dict(oaf.frequent_itemsets(data,support))  #这里设置置信度
    rules = oaf.association_rules(fre_ite,confidence)
    result=list(rules)
    return result

In [8]:
def dealResult(rules,strDecode):
    returnRules = []
    for i in rules:
        j= list(i[1])
        if len(j)==1 and strDecode[j[0]].startswith('time') :
            temStr = [];
            temp=''
            for k in i[0]:
                temp = temp+strDecode[k]+'&'
            temp = temp[:-1]
            temp = temp + ' ==> '
            temp = temp +strDecode[j[0]] 
            temStr.append(temp)
            temStr.append(i[2])
            temStr.append(i[3])
            returnRules.append(temStr)
    returnRules=list(sorted(returnRules,key=lambda x:(-x[2],-x[1])))
    return returnRules
    


In [9]:
support=0.05
confidence=0.2
train_dir='trainDataHJ.csv'
rules_dir='rulesDataHJ.txt'
data,strDecode=data_pre(train_dir)
result=model(data,support,confidence)
printRules= dealResult(result,strDecode)
file=open(rules_dir,'w')
print('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
file.write('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
for line in printRules:
    s='\t'.join([str(i) for i in line])
    print(s+'\n')
    file.write(s+'\n')
file.close()
    

Rules	support	confidence

per_con_1 ==> time_4	56	0.25925925925925924

mach_cat_2&per_con_2 ==> time_2	63	0.2581967213114754

meth_cat_1&per_con_2 ==> time_1	62	0.2572614107883817

mach_cat_2 ==> time_2	80	0.2507836990595611

is_metal_1&is_gas_0 ==> time_4	62	0.2421875

is_gas_1&per_con_2 ==> time_2	88	0.2371967654986523

tem_1 ==> time_2	79	0.23652694610778444

meth_cat_1&per_con_2 ==> time_2	57	0.23651452282157676

is_metal_1&is_gas_1 ==> time_2	54	0.23580786026200873

is_metal_1 ==> time_4	114	0.23505154639175257

per_con_2&meth_cat_3 ==> time_4	60	0.23346303501945526

meth_cat_3 ==> time_4	74	0.23270440251572327

tem_1&per_con_2 ==> time_2	60	0.23255813953488372

is_gas_1 ==> time_2	113	0.23203285420944558

hum_1&is_gas_0 ==> time_4	58	0.23107569721115537

hum_2 ==> time_2	51	0.22869955156950672

is_gas_1&is_metal_0 ==> time_2	59	0.22868217054263565

hum_1&is_metal_1 ==> time_4	59	0.2277992277992278

meth_cat_1 ==> time_2	76	0.2275449101796407

is_metal_1&is_gas_1 ==> time_4	52	0.2